In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# COIN MARKET CAP

REQUESTS

In [25]:
API_KEY = ""
with open("apikey.txt", 'r') as f:
    API_KEY = f.readline()
    
headers = {
    'X-CMC_PRO_API_KEY': API_KEY
}

def get_exchanges():
    url = 'https://pro-api.coinmarketcap.com/v1/exchange/map'
    params = {
        'sort': 'volume_24h', 
        'limit': 20
        #'category': "dex"
    }
    response = requests.get(url=url, headers=headers, params=params)
    if response.status_code == 200:
        json_data = response.json()
        return json_data
    else:
        print(response.status_code)
        return None
    
def get_exchanges_info():
    url = 'https://pro-api.coinmarketcap.com/v1/exchange/info'

    response = requests.get(url=url, headers=headers)

    if response.status_code == 200:
        json_data = response.json()
        return json_data
    else:
        print(response.status_code)
        return None
    

In [26]:
exchanges = get_exchanges()
exchanges

{'status': {'timestamp': '2024-05-01T19:24:55.109Z',
  'error_code': 0,
  'error_message': None,
  'elapsed': 12,
  'credit_count': 1,
  'notice': None},
 'data': [{'id': 270,
   'name': 'Binance',
   'slug': 'binance',
   'is_active': 1,
   'is_listed': 0,
   'is_redistributable': 1,
   'first_historical_data': '2018-04-26T00:45:00.000Z',
   'last_historical_data': '2024-05-01T19:15:00.000Z'},
  {'id': 294,
   'name': 'OKX',
   'slug': 'okx',
   'is_active': 1,
   'is_listed': 0,
   'is_redistributable': 1,
   'first_historical_data': '2018-04-26T00:45:00.000Z',
   'last_historical_data': '2024-05-01T19:15:00.000Z'},
  {'id': 1131,
   'name': 'Bitcoiva',
   'slug': 'bitcoiva',
   'is_active': 1,
   'is_listed': 0,
   'is_redistributable': 1,
   'first_historical_data': '2021-02-23T10:30:00.000Z',
   'last_historical_data': '2024-05-01T19:10:00.000Z'},
  {'id': 521,
   'name': 'Bybit',
   'slug': 'bybit',
   'is_active': 1,
   'is_listed': 0,
   'is_redistributable': 1,
   'first_histo

In [20]:
df_exchanges = pd.DataFrame(exchanges['data'])

In [21]:
df_exchanges

,id,name,slug,is_active,is_listed,is_redistributable,first_historical_data,last_historical_data
0,16,Poloniex,poloniex,1,0,1,2018-04-26T00:45:00.000Z,2024-05-01T14:35:00.000Z
1,21,BTCC,btcc,1,0,1,2018-04-26T00:45:00.000Z,2024-05-01T14:35:00.000Z
2,24,Kraken,kraken,1,0,1,2018-04-26T00:45:00.000Z,2024-05-01T14:35:00.000Z
3,34,Bittylicious,bittylicious,1,0,0,2018-04-26T00:45:00.000Z,2024-05-01T14:35:00.000Z
4,36,CEX.IO,cex-io,1,0,1,2018-04-26T00:45:00.000Z,2024-05-01T14:35:00.000Z
...,...,...,...,...,...,...,...,...
750,9867,KCEX,kcex,1,0,0,2024-04-16T10:45:00.000Z,2024-05-01T14:35:00.000Z
751,9882,Meteora VD,meteora-vd,1,0,1,2024-04-11T05:30:00.000Z,2024-05-01T14:35:00.000Z
752,9883,Meteora VP,meteora-vp,1,0,1,2024-04-11T05:30:00.000Z,2024-05-01T14:35:00.000Z
753,9933,Crypton Exchange,crypton-exchange,1,0,0,2024-04-29T10:45:00.000Z,2024-05-01T14:35:00.000Z


In [15]:
def process_of_html(soup):
    table = soup.find_all
    rows = soup.find_all('tr')
    data_list = []

    # Iterar sobre las filas de la tabla
    for row in rows[1:]:
        # Encontrar todas las celdas de la fila
        cells = row.find_all(['th', 'td'])
        cells.pop()

        # Extraer el texto de cada celda y agregarlo a la lista de datos
        data_row = []
        for cell in cells:
            # Obtener el texto de la celda
            cell_text =' '.join([text.strip() for text in cell.find_all(text=True)]) 
            class_name_found = False
            for element in cell.find_all():
                class_name = element.get('class')
                if class_name:
                    if "icon-Caret-up" in class_name:
                        cell_text = "+" + cell_text
                        class_name_found = True

                    elif "icon-Caret-down" in class_name:
                        cell_text = "-" + cell_text
                        class_name_found = True


            data_row.append(cell_text)

        data_list.append(data_row)

    return data_list

In [42]:
exchanges_table_spot = []
exchanges_table_dex = []
path_exchanges_spot = "scrap/top exchange spot.html"
path_exchanges_dex = "scrap/top exchange dex.html"
with open(path_exchanges_spot, 'r', encoding='utf-8') as archivo:
    soup = BeautifulSoup(archivo, 'html.parser')
    exchanges_table_spot = process_of_html(soup)
with open(path_exchanges_dex, 'r', encoding='utf-8') as archivo:
    soup = BeautifulSoup(archivo, 'html.parser')
    exchanges_table_dex = process_of_html(soup)

C:\Users\Cesar\AppData\Local\Temp\ipykernel_48036\1605756003.py:16: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  cell_text =' '.join([text.strip() for text in cell.find_all(text=True)])


In [49]:
clean_exchanges_spot = []
clean_exchanges_dex = []

for e in exchanges_table_spot:
    if len(e) == 9:
        clean_exchanges_spot.append(e)

for e in exchanges_table_dex:
    if len(e) == 7:
        clean_exchanges_dex.append(e)
        
print(len(clean_exchanges_spot),len(clean_exchanges_dex))


60 111


In [51]:
df_exchange_spot = pd.DataFrame(clean_exchanges_spot, columns = ['# Top', 'Exchange', 'Score', 'Trading volume(24h)', 'Avg. Liquidity', 'Weekly Visits', '# Markets', '# Coins', 'Fiat Supported'])
df_exchange_spot.head(20)

,# Top,Exchange,Score,Trading volume(24h),Avg. Liquidity,Weekly Visits,# Markets,# Coins,Fiat Supported
0,1,Binance,9.9,"$26,896,360,935",875,"16,885,328",1623,411,"EUR, GBP, BRL and +8 more"
1,2,Coinbase Exchange,8.2,"$3,851,040,529",749,"83,591",401,248,"USD, EUR, GBP"
2,3,Bybit,7.8,"$5,965,344,299",619,"7,019,422",869,610,"USD, EUR, GBP and +3 more"
3,4,OKX,7.6,"$4,590,885,421",613,"5,690,017",714,330,"AED, ARS, AUD and +43 more"
4,5,Upbit,7.4,"$3,158,192,390",540,"2,606,669",303,193,KRW
5,6,Kraken,7.3,"$1,251,528,285",750,"1,764,280",795,258,"USD, EUR, GBP and +4 more"
6,7,KuCoin,6.9,"$1,151,968,547",605,"1,964,224",1355,806,"USD, AED, ARS and +45 more"
7,8,Gate.io,6.9,"$4,080,479,573",526,"3,803,762",3586,2108,"EUR, USD, GBP and +19 more"
8,9,HTX,6.9,"$2,639,240,563",431,"4,707,746",858,700,"ALL, AUD, BRL and +47 more"
9,10,Bitfinex,6.8,"$548,175,947",535,"200,206",403,181,"USD, EUR, GBP and +1 more"


In [52]:
df_exchange_dex = pd.DataFrame(clean_exchanges_dex, columns=['# top', 'Name', 'Trading volume(24h)', '% Mkt Share', 'No. Markets', 'Type', 'Launched'])
df_exchange_dex.head(20)


,# top,Name,Trading volume(24h),% Mkt Share,No. Markets,Type,Launched
0,1,dYdX v4,"$1,627,658,249",12.6063%,62,Orderbook,Oct 2023
1,2,SynFutures v3,"$1,485,912,030",11.4565%,14,,Jun 2021
2,3,Uniswap v3 (Ethereum),"$1,337,358,501",10.3627%,966,Swap,May 2021
3,4,Jupiter,"$893,646,141",6.9149%,1046,,Oct 2021
4,5,Kine Protocol,"$858,127,910",6.6496%,56,,Mar 2021
5,6,dYdX,"$584,698,341",4.1637%,37,Orderbook,Apr 2019
6,7,Uniswap v3 (Arbitrum),"$525,752,744",4.0739%,388,,--
7,8,Vertex Protocol,"$492,051,007",3.2195%,46,Orderbook,--
8,9,Curve (Ethereum),"$463,247,799",3.5895%,138,Swap,Jan 2020
9,10,Raydium,"$460,796,475",3.5705%,954,Swap,Feb 2021


In [53]:
df_exchange_spot[:20].to_csv('dataset/top_exchange_spot.csv', sep=';')
df_exchange_dex[:20].to_csv('dataset/top_exchange_dex.csv', sep=';')

In [3]:
def get_exchange_currencies(path_html):
    
    def _process_html_table_2(soup):
        tables = soup.find_all('table')
        rows = None
        try:
            rows = tables[1].find_all('tr')
        except:
            rows = tables[0].find_all('tr')
        data_list = []
        for row in rows[1:]:
            cells = row.find_all(['th', 'td'])
            cells.pop()
            data_row = []
            for cell in cells:
                cell_text =' '.join([text.strip() for text in cell.find_all(text=True)]) 
                class_name_found = False
                for element in cell.find_all():
                    class_name = element.get('class')
                    if class_name:
                        if "icon-Caret-up" in class_name:
                            cell_text = "+" + cell_text
                            class_name_found = True
                        elif "icon-Caret-down" in class_name:
                            cell_text = "-" + cell_text
                            class_name_found = True
                data_row.append(cell_text)
            data_list.append(data_row)
        return data_list
    
    table = []
    with open(path_html, 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f, 'html.parser')
        table = _process_html_table_2(soup)
    return table

In [37]:
def get_exchange_unique_currencies(path, name):
    exchange_info = get_exchange_currencies(path)
    a = set()
    c = list()
    for e in exchange_info:
        if e[1] in a:
            continue
        else:
            a.add(e[1])
            c.append([name, 'spot', e[1], e[2].split('/')[0]])
    return c[:200]

In [38]:
exchanges_list = [    
    ['scrap/spot_exchanges/spot_binance_tr.html', 'Binance TR'],
    ['scrap/spot_exchanges/spot_binance_us.html', 'Binance US'],
    ['scrap/spot_exchanges/spot_binance.html', 'Binance'],
    ['scrap/spot_exchanges/spot_bingx.html', 'BingX'],
    ['scrap/spot_exchanges/spot_bitfinex.html', 'Bitfinex'],
    ['scrap/spot_exchanges/spot_bitflyer.html', 'bitFlyer'],
    ['scrap/spot_exchanges/spot_bitget.html', 'Bitget'],
    ['scrap/spot_exchanges/spot_bitstamp.html', 'Bitstamp'],
    ['scrap/spot_exchanges/spot_bybit.html', 'Bybit'],
    ['scrap/spot_exchanges/spot_coinbase.html', 'Coinbase'],
    ['scrap/spot_exchanges/spot_crypto_com.html', 'Crypto.com'],
    ['scrap/spot_exchanges/spot_gate_io.html', 'Gate.io'],
    ['scrap/spot_exchanges/spot_gemini.html', 'Gemini'],
    ['scrap/spot_exchanges/spot_htx.html', 'HTX'],
    ['scrap/spot_exchanges/spot_kraken.html', 'Kraken'],
    ['scrap/spot_exchanges/spot_kucoin.html', 'Kucoin'],
    ['scrap/spot_exchanges/spot_mexc.html', 'MEXC'],
    ['scrap/spot_exchanges/spot_okx.html', 'OKX'],
    ['scrap/spot_exchanges/spot_tokocrypto.html', 'Tokocrypto'],
    ['scrap/spot_exchanges/spot_upbit.html', 'Upbit']
]
currencies = []
for pair in exchanges_list:
    uniques = get_exchange_unique_currencies(pair[0], pair[1])
    currencies.extend(uniques)

C:\Users\Cesar\AppData\Local\Temp\ipykernel_20916\2732617742.py:16: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  cell_text =' '.join([text.strip() for text in cell.find_all(text=True)])


In [41]:
df_spot = pd.DataFrame(currencies, columns=['exchange', 'type', 'token', 'ticker']) 
df_spot

,exchange,type,token,ticker
0,Binance TR,spot,Tether USDt,USDT
1,Binance TR,spot,Contentos,COS
2,Binance TR,spot,Pepe,PEPE
3,Binance TR,spot,Bitcoin,BTC
4,Binance TR,spot,Bonk,BONK
...,...,...,...,...
3443,Upbit,spot,Quiztok,QTCON
3444,Upbit,spot,Juventus Fan Token,JUV
3445,Upbit,spot,Observer,OBSR
3446,Upbit,spot,AC Milan Fan Token,ACM


In [42]:
df_spot.to_csv('dataset/spot_exchange.csv', sep=';', index=False)